In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [7]:
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [10]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [11]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               'objective': 'reg:squarederror',
               'learning_rate':trial.suggest_uniform('learning_rate', 2.15e-2, 2.65e-2),
               'n_estimators':N_ESTIMATORS,
               'subsample': trial.suggest_uniform('subsample', 0.8127, 0.842),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.11, 0.13),
              'max_depth':trial.suggest_int('max_depth', 2, 4),
              'lambda':trial.suggest_uniform('lambda', 4.81, 6.31),
              'alpha':trial.suggest_uniform('alpha', 2.766, 3.766),
           'min_child_weight':trial.suggest_uniform('min_child_weight', 204, 224),
           'importance_type': 'total_gain',
            'tree_method': 'hist'
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['seed'] = inseed

    model = xgb.XGBRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-22 23:09:43,891] A new study created in memory with name: no-name-75406781-f0d6-4bee-b553-b2b5b7a42ff6


[0]	validation_0-rmse:7.57327
[1000]	validation_0-rmse:0.72720
[2000]	validation_0-rmse:0.72330
[3000]	validation_0-rmse:0.72114
[4000]	validation_0-rmse:0.71962
[5000]	validation_0-rmse:0.71847
[6000]	validation_0-rmse:0.71763
[7000]	validation_0-rmse:0.71693
[8000]	validation_0-rmse:0.71641
[9000]	validation_0-rmse:0.71597
[10000]	validation_0-rmse:0.71566
[11000]	validation_0-rmse:0.71546
[12000]	validation_0-rmse:0.71528
[13000]	validation_0-rmse:0.71517
[14000]	validation_0-rmse:0.71505
[15000]	validation_0-rmse:0.71499
[16000]	validation_0-rmse:0.71493
[17000]	validation_0-rmse:0.71488
[17334]	validation_0-rmse:0.71488
[0]	validation_0-rmse:7.57328
[1000]	validation_0-rmse:0.72740
[2000]	validation_0-rmse:0.72341
[3000]	validation_0-rmse:0.72120
[4000]	validation_0-rmse:0.71969
[5000]	validation_0-rmse:0.71853
[6000]	validation_0-rmse:0.71767
[7000]	validation_0-rmse:0.71700
[8000]	validation_0-rmse:0.71649
[9000]	validation_0-rmse:0.71604
[10000]	validation_0-rmse:0.71572
[11000

[I 2021-08-22 23:27:21,628] Trial 0 finished with value: 0.7148324547783617 and parameters: {'learning_rate': 0.026293784804099888, 'subsample': 0.8397054169915146, 'colsample_bytree': 0.12721163409054237, 'max_depth': 2, 'lambda': 6.172904569919547, 'alpha': 3.164669038972148, 'min_child_weight': 205.3262921838479}. Best is trial 0 with value: 0.7148324547783617.


[0]	validation_0-rmse:7.59155
[1000]	validation_0-rmse:0.72351
[2000]	validation_0-rmse:0.71915
[3000]	validation_0-rmse:0.71713
[4000]	validation_0-rmse:0.71599
[5000]	validation_0-rmse:0.71540
[6000]	validation_0-rmse:0.71502
[7000]	validation_0-rmse:0.71479
[8000]	validation_0-rmse:0.71467
[9000]	validation_0-rmse:0.71459
[9653]	validation_0-rmse:0.71455
[0]	validation_0-rmse:7.59155
[1000]	validation_0-rmse:0.72355
[2000]	validation_0-rmse:0.71906
[3000]	validation_0-rmse:0.71708
[4000]	validation_0-rmse:0.71602
[5000]	validation_0-rmse:0.71535
[6000]	validation_0-rmse:0.71500
[7000]	validation_0-rmse:0.71475
[8000]	validation_0-rmse:0.71455
[8957]	validation_0-rmse:0.71450


[I 2021-08-22 23:37:47,663] Trial 1 finished with value: 0.7144732102803356 and parameters: {'learning_rate': 0.023922184284137126, 'subsample': 0.8255690907817529, 'colsample_bytree': 0.1167134909462559, 'max_depth': 3, 'lambda': 4.883892254371711, 'alpha': 3.2370904024468588, 'min_child_weight': 215.29861335773373}. Best is trial 1 with value: 0.7144732102803356.


...

[0]	validation_0-rmse:7.59643
[1000]	validation_0-rmse:0.72405
[2000]	validation_0-rmse:0.71955
[3000]	validation_0-rmse:0.71741
[4000]	validation_0-rmse:0.71618
[5000]	validation_0-rmse:0.71554
[6000]	validation_0-rmse:0.71513
[7000]	validation_0-rmse:0.71487
[8000]	validation_0-rmse:0.71472
[9000]	validation_0-rmse:0.71461
[9819]	validation_0-rmse:0.71456
[0]	validation_0-rmse:7.59643
[1000]	validation_0-rmse:0.72412
[2000]	validation_0-rmse:0.71964
[3000]	validation_0-rmse:0.71750
[4000]	validation_0-rmse:0.71630
[5000]	validation_0-rmse:0.71559
[6000]	validation_0-rmse:0.71521
[7000]	validation_0-rmse:0.71491
[8000]	validation_0-rmse:0.71469
[8966]	validation_0-rmse:0.71461


[I 2021-08-23 04:42:27,349] Trial 28 finished with value: 0.7145389419843101 and parameters: {'learning_rate': 0.023288924120634963, 'subsample': 0.826557568388911, 'colsample_bytree': 0.110454590830013, 'max_depth': 3, 'lambda': 4.930673108975507, 'alpha': 2.7677058564942376, 'min_child_weight': 221.11627780568776}. Best is trial 1 with value: 0.7144732102803356.


[0]	validation_0-rmse:7.57346
[1000]	validation_0-rmse:0.72723
[2000]	validation_0-rmse:0.72331
[3000]	validation_0-rmse:0.72114
[4000]	validation_0-rmse:0.71963
[5000]	validation_0-rmse:0.71846
[6000]	validation_0-rmse:0.71763
[7000]	validation_0-rmse:0.71695
[8000]	validation_0-rmse:0.71640
[9000]	validation_0-rmse:0.71600
[10000]	validation_0-rmse:0.71571
[11000]	validation_0-rmse:0.71549
[12000]	validation_0-rmse:0.71530
[13000]	validation_0-rmse:0.71518
[14000]	validation_0-rmse:0.71506
[14623]	validation_0-rmse:0.71501
[0]	validation_0-rmse:7.57347
[1000]	validation_0-rmse:0.72740
[2000]	validation_0-rmse:0.72336
[3000]	validation_0-rmse:0.72119
[4000]	validation_0-rmse:0.71964
[5000]	validation_0-rmse:0.71844
[6000]	validation_0-rmse:0.71758
[7000]	validation_0-rmse:0.71694
[8000]	validation_0-rmse:0.71643
[9000]	validation_0-rmse:0.71600
[10000]	validation_0-rmse:0.71568
[11000]	validation_0-rmse:0.71544
[12000]	validation_0-rmse:0.71528
[13000]	validation_0-rmse:0.71513
[14000

[I 2021-08-23 04:59:53,473] Trial 29 finished with value: 0.7148720392439037 and parameters: {'learning_rate': 0.026268363756957045, 'subsample': 0.8233067682014026, 'colsample_bytree': 0.11707480726192526, 'max_depth': 2, 'lambda': 5.966595499537643, 'alpha': 3.1322038694238796, 'min_child_weight': 216.08294419840237}. Best is trial 1 with value: 0.7144732102803356.


Number of finished trials: 30
Best trial: {'learning_rate': 0.023922184284137126, 'subsample': 0.8255690907817529, 'colsample_bytree': 0.1167134909462559, 'max_depth': 3, 'lambda': 4.883892254371711, 'alpha': 3.2370904024468588, 'min_child_weight': 215.29861335773373}


In [13]:
study.best_params

{'learning_rate': 0.023922184284137126,
 'subsample': 0.8255690907817529,
 'colsample_bytree': 0.1167134909462559,
 'max_depth': 3,
 'lambda': 4.883892254371711,
 'alpha': 3.2370904024468588,
 'min_child_weight': 215.29861335773373}

# Log

====== Ordinal encoding =========

0.7191554455920611 row-wise ver2

0.718174759653254 random noise ver3

0.7151687433954103 no noise ver4

====== One-hot encoding =========

0.7146766534694031 no noise ver6

0.714641142870855 no noise ver9 (narrow space)

0.7145730004429567 no noise ver10 (narrow space)

0.7145102270468436 no noise ver11 (narrow space)

0.7145007685084899 no noise ver12 (narrow space)

0.7186947597333636 random noise ver7

0.7186322008878688 row-wise ver8
